In [9]:
import geopandas as gpd

C:\Users\clid1852\AppData\Anaconda3\envs\geoenv\lib\site-packages\geopandas\_compat.py:106: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


In [1]:
import pandas as pd

In [30]:
import requests
from bs4 import BeautifulSoup

In [1]:
import xlsxwriter

In [2]:
path = r'G:\projects\UtilityDistricts\eweb\DrinkingWater\IllegalCampCoordination\Recieved\IllegalCampNotification_pro'

In [3]:
dat = pd.read_excel(path+'\\most_recent.xlsx')

In [5]:
dat.columns

Index(['OBJECTID', 'Join_Count', 'TARGET_FID', 'Status', 'Comments', 'Date',
       'Submitted_by', 'Dogs_present', 'Unruly_inhabitants',
       'Hazardous_materials_present', 'Biohazards_present',
       'Size_of_encampment', 'maptaxlot_hyphen', 'ownname', 'addr1',
       'ownercity', 'ownerprvst', 'ownerzip', 'geocity_name', 'ugb_name',
       'longitude', 'latitude'],
      dtype='object')

In [6]:
cols2drop = ['OBJECTID', 'Join_Count', 'Unruly_inhabitants']

In [7]:
dat = dat.drop(cols2drop, axis=1)

In [8]:
intakepath = r'G:\projects\UtilityDistricts\eweb\DrinkingWater\RiparianEcosystemMarketplace\market_area\REM_area.gdb'

In [10]:
intake_areas = gpd.read_file(intakepath, driver='FileGDB', layer='AboveIntake')

In [11]:
intake_areas.crs

<Projected CRS: EPSG:2914>
Name: NAD83(HARN) / Oregon South (ft)
Axis Info [cartesian]:
- X[east]: Easting (foot)
- Y[north]: Northing (foot)
Area of Use:
- name: United States (USA) - Oregon - counties of Coos; Crook; Curry; Deschutes; Douglas; Harney; Jackson; Josephine; Klamath; Lake; Lane; Malheur.
- bounds: (-124.6, 41.98, -116.9, 44.56)
Coordinate Operation:
- name: SPCS83 Oregon South zone (International feet)
- method: Lambert Conic Conformal (2SP)
Datum: NAD83 (High Accuracy Reference Network)
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [12]:
point = gpd.read_file(path + '\\MyProject4.gdb', driver='FileGDB', layer='most_recent')

In [14]:
point = point.to_crs(epsg=2914)

In [20]:
dat.loc[:, 'TARGET_FID'] = dat.loc[:, 'TARGET_FID'] + 1085

Index(['TARGET_FID', 'Status', 'Comments', 'Date', 'Submitted_by',
       'Dogs_present', 'Hazardous_materials_present', 'Biohazards_present',
       'Size_of_encampment', 'maptaxlot_hyphen', 'ownname', 'addr1',
       'ownercity', 'ownerprvst', 'ownerzip', 'geocity_name', 'ugb_name',
       'longitude', 'latitude'],
      dtype='object')

In [48]:
point

,Status,Comments,Date,Submitted_by,Dogs_present,Unruly_inhabitants,Hazardous_materials_present,Biohazards_present,Size_of_encampment,Date_str,geometry
0,Occupied,Homeless camp,2023-01-18T22:52:35+00:00,Matthew Gilliam 541-556-1784,Unknown,None,Unknown,Unknown,Small,1/18/2023,POINT (4228098.619 897149.301)


In [61]:
url='https://services5.arcgis.com/9s1YtFmLS0YTl10F/arcgis/rest/services/ZHomeless_Camp_Trash_Collector/FeatureServer/0/1831/attachments?f=html&token='

In [62]:
#open with GET method
resp=requests.get(url)

In [64]:
if resp.status_code==200:
    soup=BeautifulSoup(resp.text,'html.parser')

In [66]:
links = []

In [67]:
for link in soup.findAll('a'):
    links.append(link.get('href'))

In [69]:
attached = [link for link in links if 'attachments' in link]

In [74]:
type(attached)

list

In [78]:
'; '.join(['https://services5.arcgis.com' + s for s in attached])

'https://services5.arcgis.com/9s1YtFmLS0YTl10F/ArcGIS/rest/services/ZHomeless_Camp_Trash_Collector/FeatureServer/0/1831/attachments/559; https://services5.arcgis.com/9s1YtFmLS0YTl10F/ArcGIS/rest/services/ZHomeless_Camp_Trash_Collector/FeatureServer/0/1831/attachments/560; https://services5.arcgis.com/9s1YtFmLS0YTl10F/ArcGIS/rest/services/ZHomeless_Camp_Trash_Collector/FeatureServer/0/1831/attachments/561'

In [85]:
datestr = points.Date.values[0].split('T')[0]

In [86]:
def convert_date(datestr):
    datestrlist = datestr.split('-')
    Y = datestrlist[0]
    m = str(int(datestrlist[1]))
    d = str(int(datestrlist[2]))
    res = m+'/'+d+'/'+Y
    return res, Y, m, d

In [43]:
'https://services5.arcgis.com' + [link for link in links if 'attachments' in link][0] 

'https://services5.arcgis.com/9s1YtFmLS0YTl10F/ArcGIS/rest/services/ZHomeless_Camp_Trash_Collector/FeatureServer/0/1832/attachments/562'

In [4]:
gdat = gpd.read_file(path + '\\MyProject4.gdb', driver='FileGDB', layer='datacopy')

In [5]:
gdat.head(3)

,Status,Comments,Date,Submitted_by,Dogs_present,Unruly_inhabitants,Hazardous_materials_present,Biohazards_present,Size_of_encampment,Date_str,geometry
0,Abandoned,Contcated group 5/18 and gave notice.,2016-05-18T15:36:59+00:00,Youngblood,No,No,None,None,Small,5/18/2016,POINT (648676.895 837520.043)
1,Cleaned-up,Cleaned up 5/18.,2016-05-18T15:38:19+00:00,Youngblood,No,No,No,No,Small,5/18/2016,POINT (648704.972 837641.883)
2,Occupied,Notified campers 5/18.,2016-05-18T15:39:57+00:00,Youngblood,No,None,None,None,Small,5/18/2016,POINT (648423.293 837925.831)


In [10]:
type(dat.Date.values[0])

str